In [8]:
!pip install requests beautifulsoup4 pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://www.1mg.com/all-diseases"
DETAIL_BASE_URL = "https://www.1mg.com"

def get_disease_list():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    disease_links = []


    for a in soup.select('a[href^="/all-diseases?label=e"]'):
        disease_name = a.get_text(strip=True)
        disease_url = DETAIL_BASE_URL + a['href']
        disease_links.append({'Disease Name': disease_name, 'Detail URL': disease_url})

    return disease_links

def get_disease_details(disease_url):
    response = requests.get(disease_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    details = {}

    def get_text_from_selector(selector):
        element = soup.select_one(selector)
        return element.get_text(strip=True) if element else None
    details['Overview'] = get_text_from_selector('.disease-content-section')
    details['Key Facts'] = get_text_from_selector('.key-facts-content')
    details['Overview'] = get_text_from_selector('.disease-content-section')

    return details

def main():

    disease_list = get_disease_list()
    print(f"Found {len(disease_list)} diseases.")


    all_disease_details = []
    for i, disease in enumerate(disease_list):
        print(f"Scraping details for {disease['Disease Name']} ({i+1}/{len(disease_list)}) from {disease['Detail URL']}")
        details = get_disease_details(disease['Detail URL'])
        all_disease_details.append({**disease, **details})
        time.sleep(1)


    df = pd.DataFrame(all_disease_details)
    df.to_csv('disease_details.csv', index=False)
    print("Data saved to disease_details.csv")
    return df
if __name__ == "__main__":
    df = main()
    print(df.head())


Found 1 diseases.
Scraping details for E (1/1) from https://www.1mg.com/all-diseases?label=e
Data saved to disease_details.csv
  Disease Name                                Detail URL Overview Key Facts
0            E  https://www.1mg.com/all-diseases?label=e     None      None
